# 传统方法：提示词工程 + 正则匹配

在 OpenAI Function Calling 出现之前，开发者通常需要依赖精心设计的提示词 (Prompt Engineering) 和正则表达式来从语言模型的输出中提取结构化信息。这种方法不仅繁琐，而且健壮性较差。

## 场景：客户信息提取系统

假设我们需要从一段用户输入的文本中提取客户的姓名、年龄、邮箱、电话和兴趣爱好。\

### 步骤1：设计提示词模板

In [1]:
# 提示词工程示例

# 提示词模板，注意JSON结构中的花括号需要双写以避免被format方法误解析为占位符
prompt_template = '''
请从以下客户描述中提取结构化信息，并严格按照JSON格式返回：

客户描述：{user_input}

请返回以下格式的JSON数据：
{{
    "name": "客户姓名",
    "age": 年龄数字,
    "email": "邮箱地址",
    "phone": "电话号码",
    "interests": ["兴趣1", "兴趣2"]
}}

注意：请确保返回的是有效的JSON格式，不要包含任何其他文字。
'''

# 示例输入
user_input = "我叫张三，今年28岁，邮箱是zhangsan@email.com，电话13812345678，喜欢编程和阅读"

# 使用format方法填充用户输入，此时JSON结构中的双花括号会被还原为单花括号
formatted_prompt = prompt_template.format(user_input=user_input)
print("--- Prompt发送给LLM ---")
print(formatted_prompt)


--- Prompt发送给LLM ---

请从以下客户描述中提取结构化信息，并严格按照JSON格式返回：

客户描述：我叫张三，今年28岁，邮箱是zhangsan@email.com，电话13812345678，喜欢编程和阅读

请返回以下格式的JSON数据：
{
    "name": "客户姓名",
    "age": 年龄数字,
    "email": "邮箱地址",
    "phone": "电话号码",
    "interests": ["兴趣1", "兴趣2"]
}

注意：请确保返回的是有效的JSON格式，不要包含任何其他文字。



In [2]:
import requests # 导入requests库用于发送HTTP请求
import json # 导入json库用于处理JSON数据

# https://dashen.zhuanspirit.com/pages/viewpage.action?pageId=512042416

# 从环境变量获取API Key，或者直接使用提供的TOKEN
# 建议实际应用中使用环境变量，这里为了演示直接使用TOKEN
# api_key = os.environ.get("DEEPSEEK_API_KEY", "YOUR_DEFAULT_TOKEN") # 更好的实践
api_key = "Mugc9IrDIlud61O0rnytgsHSPuNIyt1h" # 直接使用大神文档提供的测试 token

# 定义API的完整URL
# 注意：根据用户提供的POST示例，完整的chat completions接口是 base_url + '/chat/completions'
api_url = 'http://gptagent.zhuaninc.com/deepseek/chat/completions'
model_name = 'deepseek-chat'

base_headers = {
    "Content-Type": "application/json",
    "token": api_key # 使用提供的api_key作为 'token' 头部的值
}

payload = {
    "model": model_name,
    "messages": [
        {"role": "user", "content": formatted_prompt}
    ],
    "temperature": 0.0, # 设置温度为0，以获得更稳定的输出
}

print("--- 正在调用LLM API (使用 requests) ---")

try:
    # --- 处理非流式响应 (stream=False) ---
    # 移除流式传输相关的逻辑，只保留非流式调用
    request_headers = base_headers.copy() # 复制基础头部
    # 非流式传输需要 Accept: "application/json"
    request_headers["Accept"] = "application/json"
    # 确保 payload 中没有 stream=True (虽然在当前逻辑下不会有，但保留更安全)
    if "stream" in payload:
         del payload["stream"]
    print("--- 正在进行非流式调用 ---")

    # 发送POST请求
    response = requests.post(api_url, headers=request_headers, json=payload)
    response.raise_for_status() # 检查HTTP响应状态码，如果不是2xx则抛出异常

    # 解析JSON响应体
    response_data = response.json()

    print("--- LLM返回的原始JSON ---")
    print(json.dumps(response_data, indent=2, ensure_ascii=False)) # 打印完整的JSON响应

    # 提取模型的回复内容
    # 假设响应结构类似于OpenAI的非流式格式
    if 'choices' in response_data and len(response_data['choices']) > 0:
        # 非流式响应的内容在 message 字段中
        if 'message' in response_data['choices'][0] and 'content' in response_data['choices'][0]['message']:
            ai_response = response_data['choices'][0]['message']['content']
            print("\n--- 提取的回复内容 ---")
            print(ai_response)
        else:
             ai_response = "无法从响应中提取内容 (message/content 字段缺失)"
             print(f"\n警告: {ai_response}")
             print("完整的响应数据:", response_data)
    else:
        ai_response = "无法从响应中提取内容 (choices 字段缺失或为空)"
        print(f"\n警告: {ai_response}")
        print("完整的响应数据:", response_data)


except requests.exceptions.RequestException as e:
    # 捕获requests库可能抛出的异常 (连接错误, HTTP错误等)
    print(f"调用LLM API时发生请求错误: {e}")
    ai_response = "调用失败" # 设置一个默认值，防止后续代码出错
except Exception as e:
    # 捕获其他可能的异常
    print(f"调用LLM API时发生未知错误: {e}")
    ai_response = "调用失败" # 设置一个默认值


--- 正在调用LLM API (使用 requests) ---
--- 正在进行非流式调用 ---
--- LLM返回的原始JSON ---
{
  "id": "02174843897810015208d1b4038f629a958d0e327ef7d33d1e1ce",
  "object": "chat.completion",
  "created": 1748438980,
  "model": "deepseek-v3-241226",
  "choices": [
    {
      "index": 0,
      "message": {
        "content": "{\n    \"name\": \"张三\",\n    \"age\": 28,\n    \"email\": \"zhangsan@email.com\",\n    \"phone\": \"13812345678\",\n    \"interests\": [\"编程\", \"阅读\"]\n}",
        "role": "assistant"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 120,
    "completion_tokens": 50,
    "total_tokens": 170,
    "prompt_cache_hit_tokens": 0,
    "prompt_cache_miss_tokens": 0
  }
}

--- 提取的回复内容 ---
{
    "name": "张三",
    "age": 28,
    "email": "zhangsan@email.com",
    "phone": "13812345678",
    "interests": ["编程", "阅读"]
}
